In [ ]:
import csv
import re
from gensim.models import Word2Vec
import pandas as pd 
import numpy as np
from sklearn.datasets import load_files
from collections import Counter 
from nltk.tokenize import word_tokenize
import gc
from nltk.stem.snowball import SnowballStemmer
 
from joblib import Parallel, delayed
import multiprocessing
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.corpus import stopwords
example_sent = """This is a sample sentence,extremely"""
  
stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(example_sent) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence

['This', 'sample', 'sentence', ',', 'extremely']

In [ ]:
def majid(X,Punc=False):
    corpus = []
    for i in range(0, len(X)):
        if Punc:
          review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(X[i])) #remove punctuation
        review = re.sub(r'\d+',' ', str(X[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)             
    #Tokenizing and Word Count  
    words=[]
    for i in range(len(corpus)):
        words= nltk.word_tokenize(corpus[i])
        #sentences.append(words)
    return words

In [ ]:
path = "../data/articles1.csv"
News = pd.read_csv(path)
text= News['content']
X=text.values.tolist()
X = [[el] for el in X] 
num_cores = multiprocessing.cpu_count()
sentences = Parallel(n_jobs=num_cores)(delayed(majid)(i,True) for i in tqdm(X))

In [ ]:

"""""
Making Vocabulary and Training the Model
(sg=0 CBOW , sg=1 Skip-gram)
"""""
#########

CBOW = Word2Vec(sentences, min_count=3,size= 300,workers=multiprocessing.cpu_count(), window =1, sg = 0)
print('Done Training')

SizeOfVocab = CBOW.wv.vocab
print('Size of Vocabulary=',len(SizeOfVocab))
print('Done making the Vocabulary')

#####
Skip_gram = Word2Vec(sentences, min_count=3,size= 300,workers=multiprocessing.cpu_count(), window =1, sg = 1)
print('Done Training')

SizeOfVocab = Skip_gram.wv.vocab
print('Size of Vocabulary=',len(SizeOfVocab))
print('Done making the Vocabulary')

""""
Saving the embeddings and the model
"""""

from gensim.models import Word2Vec, KeyedVectors   
#CBOW.wv.save_word2vec_format(path+'W-CBOW-POS.bin.gz', binary=True)
path = "../Embeddings/"
CBOW.wv.save_word2vec_format(path+'W-CBOW-POS.txt', binary=False)
CBOW.save(path+'W-CBOW-POS.bin')
print('Done Saving CBOW')
#####
#Skip_gram.wv.save_word2vec_format(path+'W-Skip-POS.gz', binary=True)
Skip_gram.wv.save_word2vec_format(path+'W-Skip-POS.txt', binary=False)
Skip_gram.save(path+'W-Skip-POS.bin')
print('Done Saving Skip_gram')

print('Done Saving the Embeddings')

Done Training
Size of Vocabulary= 88803
Done making the Vocabulary
Done Training
Size of Vocabulary= 88803
Done making the Vocabulary
Done Saving CBOW
Done Saving Skip_gram
Done Saving the Embeddings
